# Forex Trading Strategy and RSI:

To identify forex trading trend, we should not rely on just the price uptrend/downtrend, but also consider the local maxima and minima over a horizon (e.g. 5 preceeding prices and 5 succeeding prices) to judge the uptrend/downtrend of the Relative Strength Indices (RSI). If both price and RSI show uptrend, then it is a real peak. If price shows uptrend but RSI shows downtrend, then it is a negative divergence. RSI is considered as a good forex trading strategy. 

**Investopedia Definition:** RSI is a momentum indicator which compares the magnitude of recent gains and losses over a specified time period to measure speed and change of price movements. It is primarily used to identify overbought or oversold conditions in the trading of an asset. 

In [1]:
# importing data
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv("../input/eurusd-forex-trading-data-20032021/EURUSD_ForexTrading_4hrs_05.05.2003_to_16.10.2021.csv")
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']

In [2]:
df=df[df['volume']!=0] #discard volume zero data points
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail() # printing tail just to check how many rows are present

,time,open,high,low,close,volume
28821,15.10.2021 01:00:00.000,1.15940,1.16120,1.15938,1.16082,5738.63
28822,15.10.2021 05:00:00.000,1.16082,1.16189,1.15995,1.16034,13149.66
28823,15.10.2021 09:00:00.000,1.16033,1.16124,1.15892,1.15923,14914.33
28824,15.10.2021 13:00:00.000,1.15923,1.16103,1.15893,1.15953,20002.53
28825,15.10.2021 17:00:00.000,1.15952,1.16060,1.15933,1.16040,5620.05


In [3]:
!pip install pandas_ta # technical analysis library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 294.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=4ecae73d5e51ef814a5eda3346e6ab83fa7b5f01c7075b83d800ca8eae95ae33
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas_ta


# Technical Analysis and Indicators

"pandas_ta" module has an inbuilt RSI indicator which uses exponential moving average instead of the absolute price. Hence it is smoothened RSI. Here, we will define our custom RSI as well, which uses the closing value of the candles. This will give more extreme values of the RSI. The objective is to check the results using both versions of RSI: smoothened and extreme. We can use the window size anything hovering between 14 to 20.

In [4]:
import numpy as np
import pandas_ta as ta # technical analysis package

# df.ta.rsi is predefined in TA module. It is "smoothened RSI". It uses exponential moving average instead of the absolute price
# 14 is the default window
df['RSI'] = df.ta.rsi(length = 14) 
df.head(15) # we will get RSI value after the 14th row since we fixed our window as 14

,time,open,high,low,close,volume,RSI
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976,NaN
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996,NaN
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992,NaN
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992,NaN
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083,NaN
5,05.05.2003 17:00:00.000,1.12718,1.13019,1.12657,1.12804,373668.2930,NaN
6,05.05.2003 21:00:00.000,1.12798,1.13004,1.12772,1.12913,94283.7988,NaN
7,06.05.2003 01:00:00.000,1.12892,1.12967,1.12743,1.12855,95461.9980,NaN
8,06.05.2003 05:00:00.000,1.12856,1.13412,1.12738,1.13381,92809.0996,NaN
9,06.05.2003 09:00:00.000,1.13383,1.13662,1.13188,1.13456,90255.7988,NaN


In [5]:
# Redefined RSI: our custom RSI which uses the closing value of the candles. This will give more extreme values of the RSI
# we are taking 20 as the window
def customRSI(price, n = 20):
    delta = price['close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(window=n).mean()
    RolDown = dDown.rolling(window=n).mean().abs()
    
    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi
df['custom_RSI'] = customRSI(df)
#df.dropna(inplace=True)
#df.reset_index(drop=True, inplace=True)

In [6]:
df.head(21) # we will get RSI value after the 20th row since we fixed our window as 20

,time,open,high,low,close,volume,RSI,custom_RSI
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976,NaN,NaN
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996,NaN,NaN
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992,NaN,NaN
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992,NaN,NaN
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083,NaN,NaN
5,05.05.2003 17:00:00.000,1.12718,1.13019,1.12657,1.12804,373668.2930,NaN,NaN
6,05.05.2003 21:00:00.000,1.12798,1.13004,1.12772,1.12913,94283.7988,NaN,NaN
7,06.05.2003 01:00:00.000,1.12892,1.12967,1.12743,1.12855,95461.9980,NaN,NaN
8,06.05.2003 05:00:00.000,1.12856,1.13412,1.12738,1.13381,92809.0996,NaN,NaN
9,06.05.2003 09:00:00.000,1.13383,1.13662,1.13188,1.13456,90255.7988,NaN,NaN


# Visualize Smoothened RSI for a Long Horizon

We will take a slice of last 825 data points to see how smoothened RSI (pandas_ta predefined RSI) looks like

In [7]:
# taking a slice from dataset

dfpl = df[28000:28825]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=dfpl.index,
    y=dfpl['RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Visualize Extreme RSI for a Long Horizon

We will take a slice of last 825 data points to see how extreme RSI (our custom version of RSI) looks like. Not much difference is visible, right?

In [8]:
# taking a slice from dataset

dfpl = df[28000:28825]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=dfpl.index,
    y=dfpl['custom_RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Visualize Smoothened RSI for a Short Horizon

Now, we will take a slice of last 100 data points to see how smoothened RSI (pandas_ta predefined RSI) looks like

In [9]:
# taking a slice from dataset

dfpl = df[28725:28825]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=dfpl.index,
    y=dfpl['RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Visualize Extreme RSI for a Short Horizon

We will take a slice of last 100 data points to see how extreme RSI (our version of custom RSI) looks like. Now, we observe some difference. If you check trace 1 curve for TA defined RSI and our redefined RSI, you will see ours one (latter one) is sharper with peaks as compared to the former (smooth) one.  

In [10]:
# taking a slice from dataset

dfpl = df[28725:28825]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=dfpl.index,
    y=dfpl['custom_RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Finding Local Maxima and Local Minima : Pivot Points

To detect the pivot points within the candle data. It will take "n" number of preceding candles of the particular candle we are interested in, and "n" number of candles coming after the one we are interested in. The objective is to identify the local minima and local maxima. Here, we are using n = 10, which means, if a candle is having higher value than its neighbouring preceding 10 candles and neighbouring succeeding 10 candles, then it is considerd as the local maxima. Similarly, if a candle is having lower value than its neighbouring preceding 10 candles and neighbouring succeeding 10 candles, then it is considerd as the local minima. 

High and low pivot ids are the peaks and dips respectively. We will define functions for finding pivot ids (peaks and dips) for both the candles and the RSIs.

In [11]:
def pivotid(df1, l, n1, n2): #check n1 before candle l, check n2 after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh: 
        return 3 # is it both pivot id high and pivot id low
    elif pividlow:
        return 1 # is it pivot id low
    elif pividhigh:
        return 2 # is it pivot id high
    else:
        return 0 # is it neither

# Define similar function for RSI pivot id
def RSIpivotid(df1, l, n1, n2): #check n1 before candle l, check n2 after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.RSI[l]>df1.RSI[i]):
            pividlow=0
        if(df1.RSI[l]<df1.RSI[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 

We are taking 10 neighbouring candles to decide local maxima and local minima, but we can experiment with some other smaller number as well.

In [12]:
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,10,10), axis=1)
df['RSIpivot'] = df.apply(lambda x: RSIpivotid(df, x.name, 10, 10), axis=1)

The following two functions are not for technical analysis, but just for ease of visualization. Point_position_x tells where to plot the pivot if it is low or high e.g. taking extreme low or high positional values like "-1e-3" and "+1e-3" respectively.

In [13]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot']==1:
        return x['RSI']-1
    elif x['RSIpivot']==2:
        return x['RSI']+1
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['RSIpointpos'] = df.apply(lambda row: RSIpointpos(row), axis=1)

# Observe Pivot Points for Long Horizon

In [14]:
# take a slice and observe pivots

dfpl = df[28000:28825]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.show()

# Observe Pivot Points for Short Horizon

In [15]:
dfpl = df[28000:28100]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)

fig.append_trace(go.Scatter(x=dfpl.index, y=dfpl['RSI']), row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=2, col=1)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

Now, we should be having both pivot and RSIpivot values stored in the dataframe itself. For convenience of plotting, we are also storing pointpos and RSIpointpos respectively.

In [16]:
df.tail(20)

,time,open,high,low,close,volume,RSI,custom_RSI,pivot,RSIpivot,pointpos,RSIpointpos
28806,12.10.2021 13:00:00.000,1.15419,1.15504,1.15254,1.15283,18731.27,33.174435,38.748242,0,1,NaN,32.174435
28807,12.10.2021 17:00:00.000,1.15283,1.15351,1.15244,1.15307,6749.27,34.656247,40.983607,1,0,1.15144,NaN
28808,12.10.2021 21:00:00.000,1.15309,1.15428,1.15292,1.15414,3025.61,40.943689,45.255474,0,0,NaN,NaN
28809,13.10.2021 01:00:00.000,1.15415,1.15559,1.15409,1.15527,4160.40,46.768937,49.843260,0,0,NaN,NaN
28810,13.10.2021 05:00:00.000,1.15528,1.15604,1.15457,1.15565,11030.24,48.604883,49.367089,0,0,NaN,NaN
28811,13.10.2021 09:00:00.000,1.15564,1.15669,1.15363,1.15430,18650.34,42.938840,48.327138,0,0,NaN,NaN
28812,13.10.2021 13:00:00.000,1.15430,1.15788,1.15405,1.15780,23598.19,56.950437,56.357928,0,0,NaN,NaN
28813,13.10.2021 17:00:00.000,1.15781,1.15975,1.15780,1.15956,11188.68,62.003143,56.426332,0,0,NaN,NaN
28814,13.10.2021 21:00:00.000,1.15961,1.16013,1.15925,1.15992,4511.04,62.960760,59.204244,0,0,NaN,NaN
28815,14.10.2021 01:00:00.000,1.15992,1.16009,1.15890,1.15900,9698.85,58.877003,53.509719,0,0,NaN,NaN


# Fitting Pivots into Slopes: Determining the Divergence of RSI

Now we will consider one "candle id" which is our candle of interest. And we will choose the number of back candles, whichis the region to consider for getting all the maximas and minimas in that regional boundary; and then we will try to fit those into the slopes to detect the divergence of the RSI.

In [17]:
import numpy as np
from matplotlib import pyplot

candleid = 1000 # our candle of interest
backcandles= 960 # fix regional boundary


maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

maximRSI = np.array([])
minimRSI = np.array([])
xxminRSI = np.array([])
xxmaxRSI = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].low)
        xxmin = np.append(xxmin, i) 
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].high)
        xxmax = np.append(xxmax, i) 
    if df.iloc[i].RSIpivot == 1:
        minimRSI = np.append(minimRSI, df.iloc[i].RSI)
        xxminRSI = np.append(xxminRSI, df.iloc[i].name)
    if df.iloc[i].RSIpivot == 2:
        maximRSI = np.append(maximRSI, df.iloc[i].RSI)
        xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)
        
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)
slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)

print(slmin, slmax, slminRSI, slmaxRSI)


dfpl = df[candleid-backcandles-5:candleid+backcandles]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'), row=1, col=1)
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'), row=1, col=1)

fig.append_trace(go.Scatter(x=dfpl.index, y=dfpl['RSI']), row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",
                marker=dict(size=2, color="MediumPurple"),
                name="pivot", row=2, col=1)
fig.add_trace(go.Scatter(x=xxminRSI, y=slminRSI*xxminRSI + intercminRSI, mode='lines', name='min slope'), row=2, col=1)
fig.add_trace(go.Scatter(x=xxmaxRSI, y=slmaxRSI*xxmaxRSI + intercmaxRSI, mode='lines', name='max slope'), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

4.745677335753135e-05 3.26015746678941e-05 0.015011532795376769 0.012114176551967752


**Observation :** We see two slopes (maxima slope which denotes uptrend and minima slope which denotes downtrend) for both candles i.e. prices (upper chart) and RSIs (lower chart). Hence, it is validated that the functions are working fine to detect uptrend and downtrend for both the candles and the RSIs. Now, if we want to use this for trading, we need to generate a signal for the divergence. 

# Generate Divergence Signal

From above we can get the combinations 
- Case 1 - Price is showing uptrend, RSI is also showing uptrend : Then it is an uptrend without divergence
- Case 2 - Price is showinf uptrend, but RSI is showing downtrend : Then it is actually a **negative divergence**
- Case 3 - Price is showing downtrend, RSI is also showing downtrend : Then it is a downtrend without divergence
- Case 4 - Price is showing downtrend, RSI is also showing uptrend : Then it is actually a **positive divergence**

In [18]:
dfpl = df[0:5000] # taking a slice of 5000 datapoints
def divsignal(x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size<2 or minim.size<2 or maximRSI.size<2 or minimRSI.size<2:
        return 0
    
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)
    
    
    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI <-0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0

dfpl['divSignal'] = dfpl.apply(lambda row: divsignal(row,30), axis=1) # check 30 neighbouring candles to get minima and maxima 

Checking the divergence signals in the dataframe

In [19]:
dfpl.tail()

,time,open,high,low,close,volume,RSI,custom_RSI,pivot,RSIpivot,pointpos,RSIpointpos,divSignal
4995,12.07.2006 09:00:00.000,1.27471,1.27522,1.26829,1.27130,146188.3984,39.335591,41.001309,0,0,NaN,NaN,0
4996,12.07.2006 13:00:00.000,1.27104,1.27181,1.26788,1.26970,485854.8986,36.273033,37.801014,0,0,NaN,NaN,0
4997,12.07.2006 17:00:00.000,1.26976,1.27064,1.26866,1.27035,559858.2969,38.372230,29.540560,0,0,NaN,NaN,0
4998,12.07.2006 21:00:00.000,1.27028,1.27136,1.26886,1.27039,147971.5000,38.506472,27.830882,0,0,NaN,NaN,0
4999,13.07.2006 01:00:00.000,1.27043,1.27225,1.27032,1.27114,152179.4024,41.097282,31.431810,0,0,NaN,NaN,0


Checking how many negative and positive divergence are present in our chosen 5000 datapoints

In [20]:
dfpl[dfpl.divSignal==1].count()

time           12
open           12
high           12
low            12
close          12
volume         12
RSI            12
custom_RSI     12
pivot          12
RSIpivot       12
pointpos        0
RSIpointpos     1
divSignal      12
dtype: int64

In [21]:
dfpl[dfpl.divSignal==2].count()

time           5
open           5
high           5
low            5
close          5
volume         5
RSI            5
custom_RSI     5
pivot          5
RSIpivot       5
pointpos       2
RSIpointpos    1
divSignal      5
dtype: int64

We got 12 negative divergence points and 5 positive divergence points in first 5000 datapoints.

In [22]:
#Export the results
dfpl.to_csv('divergence_signal_results.csv', index=False)

# Acknowledgement :

I learnt the entire methodology and the code for functions implementation from [Code Trading Cafe ](https://www.youtube.com/c/CodeTradingCafe/about)